<a href="https://colab.research.google.com/github/O0O0O0O0OP48763/123/blob/master/DG_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import scipy.special as sp
from numpy.polynomial.legendre import leggauss

def compute_integral(n_intervals, poly_order):
    """
    計算積分 I_{ij}^n = ∫_{I^n} φ_j(t) φ_i'(t) dt 使用 Gauss-Legendre 積分

    :param n_intervals: 區間數量 N
    :param poly_order: Legendre 多項式最高階數
    :return: I 矩陣 (n_intervals, poly_order+1, poly_order+1)
    """
    x_nodes = np.linspace(0, 2, n_intervals + 1)  # 劃分區間節點
    I_matrix = np.zeros((n_intervals, poly_order + 1, poly_order + 1))  # 儲存每個區間的積分矩陣

    # Gauss-Legendre 積分點與權重
    gauss_x, gauss_w = leggauss(poly_order + 1)  # 使用 (poly_order+1) 個積分點

    for k in range(n_intervals):
        x_k, x_kp1 = x_nodes[k], x_nodes[k + 1]
        h = x_kp1 - x_k  # 區間長度

        for i in range(poly_order + 1):
            P_i = sp.legendre(i)  # P_i(x)
            dP_i = P_i.deriv()    # P_i'(x)

            for j in range(poly_order + 1):
                P_j = sp.legendre(j)  # P_j(x)

                # 使用 Gauss-Legendre 積分
                integral = 0.0
                for q in range(len(gauss_x)):
                    xi = gauss_x[q]  # [-1,1] 上的積分點
                    t = x_k + (xi + 1) * h / 2  # 對應回 [x_k, x_kp1]
                    phi_j = P_j(xi)  # P_j(T(t))
                    phi_i_prime = (2 / h) * dP_i(xi)  # d/dt[P_i(T(t))] = (2/h) P_i'(ξ)
                    integral += gauss_w[q] * phi_j * phi_i_prime  # 加總加權積分值

                integral *= h / 2  # 調整積分區間對應的權重

                I_matrix[k, i, j] = integral  # 儲存結果

    return I_matrix

# 測試
n_intervals = 6  # 4 個子區間
poly_order = 3   # 最高階為 2 的 Legendre 多項式
I_result = compute_integral(n_intervals, poly_order)

# 顯示計算結果
print("積分矩陣 I_{ij}^n:")
for k in range(n_intervals):
    print(f"區間 {k+1}:")
    print(I_result[k])


積分矩陣 I_{ij}^n:
區間 1:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.00000000e+00 -3.70074342e-17 -3.33066907e-16  1.11022302e-16]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -2.96059473e-16]
 [ 2.00000000e+00  0.00000000e+00  2.00000000e+00 -2.22044605e-16]]
區間 2:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.00000000e+00 -3.70074342e-17 -3.33066907e-16  1.11022302e-16]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -2.96059473e-16]
 [ 2.00000000e+00  0.00000000e+00  2.00000000e+00 -2.22044605e-16]]
區間 3:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.00000000e+00  3.70074342e-17 -3.33066907e-16  7.40148683e-17]
 [ 0.00000000e+00  2.00000000e+00  0.00000000e+00 -2.22044605e-16]
 [ 2.00000000e+00  0.00000000e+00  2.00000000e+00 -2.22044605e-16]]
區間 4:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.00000000e+00  0.00000000e+00 -2.96059473e-16  5.55111512e-17]
 [ 0.00000000e+00  2

In [5]:
import numpy as np
import scipy.special as sp
from numpy.polynomial.legendre import leggauss

def compute_integral_f_phi(n_intervals, poly_order, f):
    """
    計算 I_i^n = ∫_{I^n} f(t) φ_i(t) dt 使用 Gauss-Legendre 積分

    :param n_intervals: 區間數量
    :param poly_order: Legendre 多項式最高階數
    :param f: 目標函數 f(t)
    :return: I 矩陣 (n_intervals, poly_order+1)
    """
    x_nodes = np.linspace(0, 2, n_intervals + 1)  # 劃分區間
    I_matrix = np.zeros((n_intervals, poly_order + 1))  # 存儲積分結果

    # Gauss-Legendre 積分點與權重
    gauss_x, gauss_w = leggauss(poly_order + 1)  # 使用 (poly_order+1) 個積分點

    for k in range(n_intervals):
        x_k, x_kp1 = x_nodes[k], x_nodes[k + 1]
        h = x_kp1 - x_k  # 區間長度

        # 變換回原本區間 I^n 的函數
        t_values = 0.5 * h * (gauss_x + 1) + x_k  # T^{-1}(ξ)

        for i in range(poly_order + 1):
            P_i_values = sp.legendre(i)(gauss_x)  # 計算 φ_i(ξ)
            integral = np.sum(gauss_w * f(t_values) * P_i_values) * (h / 2)  # Gauss-Legendre 積分
            I_matrix[k, i] = integral  # 儲存結果

    return I_matrix

# 測試
n_intervals = 4  # 4 個子區間
poly_order = 2   # 最高階為 2 的 Legendre 多項式
f = lambda t: np.exp(t)  # f(t) = e^t
I_result = compute_integral_f_phi(n_intervals, poly_order, f)

# 顯示計算結果
print("積分結果 I_i^n:")
for k in range(n_intervals):
    print(f"區間 {k+1}: {I_result[k]}")


積分結果 I_i^n:
區間 1: [0.64872127 0.05383607 0.00268342]
區間 2: [1.06956055 0.08876067 0.00442422]
區間 3: [1.76340723 0.14634161 0.0072943 ]
區間 4: [2.90736701 0.24127652 0.01202627]
